# Mixed IR/SE
This document attempts to describe the procedure for estimation of $T_1$-time within cerebrospinal fluid based on the interleaved inversion-recovery, spin-echo (Mixed IR/SE) MRI-data for fast $T_1$-time estimates within cerebrospinal fluid (CSF).
Note that these steps are further detailed in the article [https://doi.org/10.1101/2025.07.19.25331835], which is related to the work done for the Gonzo data record.

The steps include:
- Converting necessary volumes from enhanced DICOM to Nifti.
- Running FreeSurfer on a pre-contrast $T_1$-weighted image --- potentially supported by either FLAIR or a $T_2$-weighted image --- for the purpose of extrapolating FreeSurfer brain segmentations to the CSF-space.
- Estimating the $T_1$-time by using a lookup-table for the ratio between the inversion recovery and spin-echo signal.
- Registering the Mixed IR/SE images to the reference space of the pre-contrast $T_1$-weighted image.

**Data**:

To replicate the processing steps of the accompanying article, you need the following data for each subject, (with `ses` representing the session):
- `mri_dataset/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii.gz`
- `mri_dataset/sub-01/$ses/mixed/sub-01_${ses}_acq-mixed_IR-corrected-real.nii.gz`
- `mri_dataset/sub-01/$ses/mixed/sub-01_${ses}_acq-mixed_SE-modulus.nii.gz`
- `mri_dataset/sub-01/$ses/mixed/sub-01_${ses}_acq-mixed_meta.json`
- (`mri_dataset/sub-01/ses-01/anat/sub-01_ses-01_FLAIR.nii.gz`)

The `sub-01_${ses}_acq-mixed_meta.json`-files contain information about repetition times and echo times for both the IR and SE images, and are created as part of the DICOM-conversion process. See section [DICOM conversion](#DICOM-conversion) for details on the conversion from DICOM to Nifti. 
Note that the FLAIR-image is only used for supporting the FreeSurfer `recon-all` pipeline, and is not strictly necessary.

## Executing pipeline
Asssuming that all data dependencies exist, the easiest way to run the pipeline for a given subject, is to execute
```bash
pixi run snakemake -p -c 'all'
```
which uses Snakemake to figure out the necessary rules to run, and executes them. More details on the processing steps are given below.
**NB**: If you do not have access to FreeSurfer or Greedy on your computer, snakemake may be configured to run these steps in a Singularity container, if you add the option `--profile profile` to the snakemake-call. This requires you to have FreeSurfer license file located in the root directory.


## Pipeline description

### Running FreeSurfer
FreeSurfer is used to create a base segmentation of the brain, which in turn is extrapolated to the subarachnoid space and ventricles. FreeSurfer may be run by:
```bash
recon-all -all  \
    -s sub-01 \
    -sd mri_processed_data/freesurfer \
    -FLAIR mri_dataset/sub-01/ses-01/anat/sub-01_ses-01_FLAIR.nii.gz \
    -FLAIRpial \
    -parallel 
```
**PS**: This step may take somewhere between a few hours to a day, depending on your computer and the image quality . For the purpose of extrapolating segments, you could also consider to use the much faster deep-learning-based `FastSurfer`, which is a drop-in replacement for FreeSurfer.
Any potential differences between the segmentations should (probably) have minimal impact for the analysis of the mixed-article.

### Estimating $T_1$-times.
The $T_1$-maps of the mixed sequence are estimated by comparing the voxelwise ratio of the signal intenstities to a look-up table. The command to generate the mixed T1maps is

```bash
gmri2fem mri mixed-t1map \
    --SE mri_dataset/sub-01/$ses/mixed/sub-01_${ses}_acq-mixed_SE-modulus.nii.gz \
    --IR mri_dataset/sub-01/$ses/mixed/sub-01_${ses}_acq-mixed_IR-corrected-real.nii.gz \
    --meta mri_dataset/sub-01/$ses/mixed/sub-01_${ses}_acq-mixed_meta.json \
    --output mri_dataset/derivatives/sub-01/$ses/sub-01_${ses}_acq-mixed_T1map_raw.nii.gz
```

For details on how this is done, see (https://github.com/jorgenriseth/gMRI2FEM/blob/f11a9961b3689250d7fd6cf41357dd7d99a53f68/src/gmri2fem/mixed_t1map.py).

### Registering to common image space
For longitudinal analysis of the images taken of the same subject at different sessions, we need to register them all to a common image space.
For this purpose we use the software `greedy` which is bundled with ITK-snap, or can be downloaded separately.
We are only interested in registering images for the same subject, and therefore only consider rigid registration between the sessions.
As the reference-image we use the pre-contrast $T_1$-weighted image, which is used by FreeSurfer for the original brain segmentation.

`greedy` works in a two-step process: First the registration procedure is applied to find the best – in this case, affine – transformation mapping the data from a moving image (the mixed-images) to the fixed image (the $T_1$-weighted image).
The transformation is outputted as a separate text-file as a transformation matrix using homogeneneous coordinates.
This transformation may in turn be used to "reslice" the moving image (and any images aligned with it) into the image space of the target image.

For registering the Mixed IR/SE images, we use the following command to register the pre-contrast SE-modulus image to the reference image:
```bash
greedy -d 3 -a \
 -i mri_dataset/sub-01/ses-01/anat/sub-01_$ses-01_T1w.nii.gz \
     mri_dataset/sub-01/ses-01/mixed/sub-01_ses-01_acq-mixed_SE-modulus.nii.gz \
 -o mri_processed_data/sub-01/transforms/{subject}_{session}_acq-mixed.mat \
 -ia-image-centers \
 -dof 6 \
 -m NMI
```

```bash
greedy -d
 -rf mri_dataset/sub-01/ses-01/anat/sub-01_$ses-01_T1w.nii.gz \
 -ri NN \
 -rm mri_dataset/sub-01/ses-01/mixed/sub-01_ses-01_acq-mixed_SE-modulus.nii.gz \
     mri_processed_data/sub-01/registered/sub-01_ses-01_acq-mixed_SE-modulus_registered.nii.gz
 -r mri_processed_data/sub-01/transforms/{subject}_{session}_acq-mixed.mat
```

The SE-modulus images from the remaining sessions are in turn registered to the already registered pre-contrast SE-modulus image by
```bash
greedy -d 3 -a \
 -i mri_processed_data/sub-01/registered/sub-01_ses-01_acq-mixed_SE-modulus_registered.nii.gz \ 
     mri_dataset/sub-01/$ses/mixed/sub-01_${ses}_acq-mixed_SE-modulus.nii.gz \
 -o mri_processed_data/sub-01/transforms/{subject}_{session}_acq-mixed.mat \
 -ia-image-centers \
 -dof 6 \
 -m NCC 5x5x5
```

Finally each of the transformations are applied to the corresponding $T_1$-map:
```bash
greedy -d 3
 -rf mri_dataset/sub-01/ses-01/anat/sub-01_$ses-01_T1w.nii.gz \
 -ri NN \
 -rm mri_dataset/derivatives/sub-01/$ses/sub-01_ses-01_acq-mixed_T1map_raw.nii.gz \
     mri_processed_data/sub-01/registered/sub-01_ses-01_acq-mixed_T1map_raw_registered.nii.gz
 -r mri_processed_data/sub-01/transforms/{subject}_{session}_acq-mixed.mat
```


## DICOM Conversion

We prefer to work with Nifti-images over DICOM-images due to concerns related to anonymization, storage effieciency, generalizability, and simplicity.
Note that due to differences between project structure, MRI-machine vendor etc., this procedure is specific to our project, and might not be directly applicable for a different project. We still include a description of it for transparency, and for use as a starting point for other projects.

Most DICOM-images, including the $T_1$-weighted and FLAIR images, may safely be  converted from Enhanced DICOM format to Nifti using the tool `dcm2niix` [https://github.com/rordenlab/dcm2niix].
For the Mixed IR/SE-sequence, however, it converts the data in an inconcistent way, where it ocasionally outputs all six volumes as expected, but at other times, it outputs fewer number of images where some of them might contain two volumes as one. 
We have therefore written a custom script for converting mixed-sequences from DICOM to Nifti.
The script is available on (https://github.com/jorgenriseth/gMRI2FEM/blob/f11a9961b3689250d7fd6cf41357dd7d99a53f68/src/gmri2fem/mixed_dicom.py), and may be called 
as follows.

```bash
gmri2fem dcm2nii-mixed --subvolume IR-corrected-real --subvolume SE-modulus \
    path/to/dicom/file/for/subject/session/IM_XXXX \
    mri_dataset/$subject/$session/mixed/${subject}_${session}_acq-mixed
```